In [1]:
using JuMP
using Ipopt
solver = IpoptSolver()

Ipopt.IpoptSolver(Any[])

In [2]:
# Problem definition
nctrl = 10; # number of control points
nt = 5; # number of time steps
r = 1; # radius of the motor
circ = 2*pi*r; # maximum circumference of the motor
circProfile = pi*r*ones(nt);
regrate = 0.05;

In [5]:
function soromotor_model(solver, nctrl::Int, nt::Int, r, regrate,circProfile)
    m = Model(solver = solver)
    
    # variables
    @variables m begin
        x[1:nctrl, 1:nt]
        y[1:nctrl, 1:nt]    
        nx[1:nctrl, 1:nt]
        ny[1:nctrl, 1:nt]
        l[1:nctrl, 1:nt]
        circ[1:nt]
    end
            
    
    # constraints
    @constraints m begin
        l[:,:] .>= 0
        circ[:,:] .>= 0
        # Motion of flame front 
        x[1:nctrl,2:nt] .== x[1:nctrl,1:nt-1] + 
            0.5*nx[1:nctrl,1:nt-1]*regrate + 
            0.5*nx[1:nctrl,1:nt-1]*regrate
        y[1:nctrl,2:nt] .== y[1:nctrl,1:nt-1] + 
            0.5*ny[1:nctrl,1:nt-1]*regrate + 
            0.5*ny[1:nctrl,1:nt-1]*regrate
    end
    # nonlinear constraints
    for i in 1:nctrl
        for j in 1:nt
            @NLconstraint(m, x[i,j]^2 + y[i,j]^2 <= r^2)
        end
    end
    for i in 2:nctrl
        for j in 1:nt
            @NLconstraint(m, ny[i,j] == (x[i,j]-x[i-1,j])/l[i,j])
            @NLconstraint(m, nx[i,j] == -(y[i,j]-y[i-1,j])/l[i,j])
            @NLconstraint(m, l[i,j] == (x[i,j] - x[i-1,j])^2 + 
                (y[i,j] - y[i-1,j])^2)
        end
    end
    for j in 1:nt      
        @NLconstraint(m, ny[1,j] == (x[1,j]-x[nctrl,j])/l[1,j])      
        @NLconstraint(m, nx[1,j] == -(y[1,j]-y[nctrl,j])/l[1,j])
        @NLconstraint(m, l[1,j] == (x[1,j] - x[nctrl,j])^2 + (y[1,j] - y[nctrl,j])^2)
    end
    for i in 1:nt
        @constraint(m, circ[i] == sum(x for x in l[1:nctrl,i]))
    end
    @NLobjective(m, Min, sum((circ[a]-circProfile[a])^2 for a in 1:nt))
    return m
end



soromotor_model (generic function with 1 method)

In [6]:
m = soromotor_model(solver, nctrl, nt, r, regrate,circProfile)
solve(m)
#status = solve(m)

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:      890
Number of nonzeros in inequality constraint Jacobian.:      210
Number of nonzeros in Lagrangian Hessian.............:     1005

The Jacobian for the equality constraints contains an invalid number

Number of Iterations....: 0

Number of objective function evaluations             = 0
Number of objective gradient evaluations             = 0
Number of equality constraint evaluations            = 0
Number of inequality constraint evaluations          = 1
Number of equality constraint Jacobian evaluations   = 1
Number of inequality constraint Jacobian evaluations = 0
Number of Lagrangian Hessian evaluations             = 0
Total CPU secs in IPOPT (w/o function evaluations)   =      0.000
Total CPU secs in NLP function evaluations           =      0.000

EXIT: Invalid number in NLP funct

:Error